# Analisys

In [4]:
import json
import os
import math
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import logging
import numpy as np  # Para isnan
from typing import Dict, List, Any, Optional

# Configuración básica de logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- Constantes ---
CTI = 0.2  # Intervalo de tiempo de coordinación típico (en segundos)

# --- Rutas de los archivos ---
INPUT_FILE = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_paper.json"
# PLOT_OUTPUT_PATH = "/Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/model/tmt_analysis_all_scenarios_from_input_times.html"

logger.info(f"Archivo de entrada: {INPUT_FILE}")

# %%
# Celda 2: Funciones de Ayuda (Carga)
def load_json_file(file_path: str) -> Optional[Any]:
    """Carga datos desde un archivo JSON."""
    try:
        with open(file_path, 'r') as file:
            data = json.load(file)
        logger.info(f"Archivo cargado: {file_path}")
        return data
    except FileNotFoundError:
        logger.error(f"No encontrado: {file_path}")
        return None
    except json.JSONDecodeError as e:
        logger.error(f"JSON inválido: {file_path}: {e}")
        return None
    except Exception as e:
        logger.error(f"Error carga {file_path}: {e}")
        return None

# %%
# Celda 3: Carga y Agrupación de Datos por Escenario
logger.info("Cargando y agrupando datos por escenario...")
relay_pairs_data = load_json_file(INPUT_FILE)

if relay_pairs_data is None:
    raise SystemExit("Error crítico al cargar archivo de entrada.")
if not isinstance(relay_pairs_data, list):
    raise SystemExit("Error de formato en archivo de entrada (se espera lista).")

original_pairs_map: Dict[str, List[Dict]] = {}
total_original_pairs_read = 0
skipped_pairs_structure = 0

for pair in relay_pairs_data:
    s_id = pair.get("scenario_id")
    main_info = pair.get('main_relay')
    backup_info = pair.get('backup_relay')
    if s_id and isinstance(main_info, dict) and isinstance(backup_info, dict) and \
       'Time_out' in main_info and 'Time_out' in backup_info:
        original_pairs_map.setdefault(s_id, []).append(pair)
        total_original_pairs_read += 1
    else:
        skipped_pairs_structure += 1

logger.info(f"Datos agrupados: {len(original_pairs_map)} escenarios encontrados.")
logger.info(f"{total_original_pairs_read} pares válidos leídos y agrupados.")
if skipped_pairs_structure > 0:
    logger.warning(f"{skipped_pairs_structure} pares fueron omitidos durante la carga debido a estructura o Time_out faltante.")

# %%
# Celda 4: Calcular Métricas de Coordinación para Todos los Escenarios
logger.info("Calculando TMT y métricas para cada escenario usando los Time_out existentes...")
scenario_results: Dict[str, Dict[str, Any]] = {}

if not original_pairs_map:
    raise SystemExit("No hay datos de escenario para procesar.")

for scenario_id, pairs_for_scenario in original_pairs_map.items():
    tmt_for_scenario = 0.0
    miscoordination_count = 0
    valid_pairs_evaluated = 0
    max_neg_mt_scenario = 0.0
    skipped_this_scenario = 0

    for pair_entry in pairs_for_scenario:
        main_time = pair_entry['main_relay'].get('Time_out')
        backup_time = pair_entry['backup_relay'].get('Time_out')

        if not isinstance(main_time, (int, float)) or math.isnan(main_time) or \
           not isinstance(backup_time, (int, float)) or math.isnan(backup_time):
            skipped_this_scenario += 1
            continue

        delta_t = backup_time - main_time - CTI
        mt = min(0, delta_t)

        valid_pairs_evaluated += 1
        if mt < -1e-6:
            miscoordination_count += 1
            tmt_for_scenario += mt
            max_neg_mt_scenario = min(max_neg_mt_scenario, mt)

    if valid_pairs_evaluated > 0:
        scenario_results[scenario_id] = {
            "TMT": tmt_for_scenario,
            "MiscoordinatedCount": miscoordination_count,
            "TotalPairsEvaluated": valid_pairs_evaluated,
            "MaxNegativeMT": max_neg_mt_scenario
        }
        if skipped_this_scenario > 0:
            logger.warning(f"({scenario_id}) Se omitieron {skipped_this_scenario} pares adicionales por tiempos inválidos.")
    elif pairs_for_scenario:
        logger.warning(f"({scenario_id}) No se evaluaron pares válidos (problemas con Time_out?).")

logger.info(f"Cálculo de TMT completado para {len(scenario_results)} escenarios.")

# --- Nuevo bloque: imprimir TMT de cada escenario ---
logger.info("Valores de TMT por escenario:")
for scenario_id, metrics in scenario_results.items():
    tmt = metrics.get("TMT", 0.0)
    print(f"Escenario {scenario_id}: TMT = {tmt:.4f} s")

# %%
# Celda 5: Análisis - Mostrar los 54 Mejores Escenarios por TMT
if not scenario_results:
    logger.error("No hay resultados de escenario para analizar.")
else:
    df_results = pd.DataFrame.from_dict(scenario_results, orient='index')
    df_results.index.name = 'scenario_id'
    df_results = df_results.reset_index()
    df_results_sorted_by_tmt = df_results.sort_values(by='TMT', ascending=False)
    top_54_scenarios = df_results_sorted_by_tmt[df_results_sorted_by_tmt['TMT'] != 0].head(54)

    print("\n--- Mejores 54 Escenarios por TMT (Basado en Tiempos del Archivo de Entrada) ---")
    pd.set_option('display.max_rows', 60)
    print(top_54_scenarios[['scenario_id', 'TMT', 'MiscoordinatedCount', 'TotalPairsEvaluated', 'MaxNegativeMT']].round(5))
    pd.reset_option('display.max_rows')

# %%
# Celda 6: Visualización - Gráfico de Barras del TMT para Todos los Escenarios
if scenario_results and 'df_results' in locals():
    logger.info("Generando gráfico de barras de TMT para todos los escenarios...")

    def get_scenario_num(s_id):
        try:
            return int(s_id.split('_')[-1])
        except:
            return float('inf')

    df_results_plot_sorted = df_results.copy()
    df_results_plot_sorted['scenario_num'] = df_results_plot_sorted['scenario_id'].apply(get_scenario_num)
    df_results_plot_sorted = df_results_plot_sorted.sort_values(by='scenario_num')

    scenario_labels_plot = df_results_plot_sorted['scenario_id'].tolist()
    tmt_values_plot    = df_results_plot_sorted['TMT'].tolist()
    miscoord_counts_plot = df_results_plot_sorted['MiscoordinatedCount'].tolist()
    total_pairs_plot     = df_results_plot_sorted['TotalPairsEvaluated'].tolist()
    max_neg_mt_plot      = df_results_plot_sorted['MaxNegativeMT'].tolist()

    hover_texts_plot = [
        f"<b>{s_id}</b><br>TMT: {tmt:.4f}<br>Descoordinados: {mc}/{tp}<br>Peor MT: {max_neg_mt:.4f}"
        for s_id, tmt, mc, tp, max_neg_mt in zip(scenario_labels_plot, tmt_values_plot, miscoord_counts_plot, total_pairs_plot, max_neg_mt_plot)
    ]

    colors = [
        '#2ca02c' if tmt >= -0.001 else
        '#ff7f0e' if tmt >= -0.05 else
        '#d62728'
        for tmt in tmt_values_plot
    ]

    fig_tmt_all = go.Figure(data=[
        go.Bar(
            x=scenario_labels_plot,
            y=tmt_values_plot,
            text=[f"{mc}/{tp}" for mc, tp in zip(miscoord_counts_plot, total_pairs_plot)],
            textposition='outside',
            textfont_size=8,
            hovertext=hover_texts_plot,
            hoverinfo='text',
            marker_color=colors
        )
    ])

    fig_tmt_all.update_layout(
        title="TMT Total por Escenario (Basado en Tiempos del Archivo de Entrada)",
        xaxis_title="ID de Escenario",
        yaxis_title="TMT Total (s)",
        yaxis=dict(range=[min(tmt_values_plot + [-0.1]) * 1.1, max(tmt_values_plot + [0.05])]),
        yaxis_zeroline=True, yaxis_zerolinewidth=2, yaxis_zerolinecolor='black',
        xaxis={'type': 'category', 'tickangle': -90},
        height=700,
        bargap=0.2,
        hovermode='x unified'
    )

    fig_tmt_all.show()
logger.info("--- Análisis de TMT completado ---")


2025-04-19 23:42:55,419 - INFO - Archivo de entrada: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_paper.json
2025-04-19 23:42:55,423 - INFO - Cargando y agrupando datos por escenario...
2025-04-19 23:42:55,478 - INFO - Archivo cargado: /Users/gustavo/Documents/Projects/TESIS_UNAL/ADAPTIVE_ALGORITHM/data/processed/independent_relay_pairs_paper.json
2025-04-19 23:42:55,526 - INFO - Datos agrupados: 68 escenarios encontrados.
2025-04-19 23:42:55,527 - INFO - 6800 pares válidos leídos y agrupados.
2025-04-19 23:42:55,527 - INFO - Calculando TMT y métricas para cada escenario usando los Time_out existentes...
2025-04-19 23:42:55,539 - INFO - Cálculo de TMT completado para 68 escenarios.
2025-04-19 23:42:55,542 - INFO - Valores de TMT por escenario:
2025-04-19 23:42:55,651 - INFO - Generando gráfico de barras de TMT para todos los escenarios...


Escenario scenario_1: TMT = -20.0000 s
Escenario scenario_2: TMT = -20.0000 s
Escenario scenario_3: TMT = -20.0000 s
Escenario scenario_4: TMT = -20.0000 s
Escenario scenario_5: TMT = -20.0000 s
Escenario scenario_6: TMT = -20.0000 s
Escenario scenario_7: TMT = -20.0000 s
Escenario scenario_8: TMT = -20.0000 s
Escenario scenario_9: TMT = -20.0000 s
Escenario scenario_10: TMT = -20.0000 s
Escenario scenario_11: TMT = -20.0000 s
Escenario scenario_12: TMT = -20.0000 s
Escenario scenario_13: TMT = -20.0000 s
Escenario scenario_14: TMT = -20.0000 s
Escenario scenario_15: TMT = -20.0000 s
Escenario scenario_16: TMT = -20.0000 s
Escenario scenario_17: TMT = -20.0000 s
Escenario scenario_18: TMT = -20.0000 s
Escenario scenario_19: TMT = -20.0000 s
Escenario scenario_20: TMT = -20.0000 s
Escenario scenario_21: TMT = -20.0000 s
Escenario scenario_22: TMT = -20.0000 s
Escenario scenario_23: TMT = -20.0000 s
Escenario scenario_24: TMT = -20.0000 s
Escenario scenario_25: TMT = -20.0000 s
Escenario

2025-04-19 23:42:55,990 - INFO - --- Análisis de TMT completado ---
